# Classification of Cassava Leaves

## Resnet50 Architecture

### Imports

In [3]:
import os  # for path operations
import pickle  # for loading created data
import time  # calculating durations

import numpy as np  # for the matrix operations
from tensorflow.keras.applications import ResNet50  # Resnet50 layer architecture
from tensorflow.keras.layers import Input  # input layer that feeds main architecture
from tensorflow.keras.losses import SparseCategoricalCrossentropy  # loss function
from tensorflow.keras.optimizers import Adam  # used optimizer

### Definiation of parameters

In [4]:
learning_rate = 0.01  # model learning rate
# learning_rates = 0.1  # other learning rate

val_split = 0.2  # validation data split for model

IMG_W, IMG_H, CHAN = 200, 200, 3  # input data parameters 200x200 pixels and 3 color channels as RGB

# below path should be edit
input_data_path = ""  # this directory should be the output_data_path of aug_image_editing script
output_data_path = ""

train_data_path = os.path.join(input_data_path, f"train_data/aug_{IMG_W}x{IMG_H}x{CHAN}")
test_data_path = os.path.join(input_data_path, f"test_data/aug_{IMG_W}x{IMG_H}x{CHAN}")

### Loading input data

In [ ]:
with open(os.path.join(train_data_path, "train.pickle"), "rb") as f:
    X_train = pickle.load(f)  # loading train image data
with open(os.path.join(train_data_path, "label.pickle"), "rb") as f:
    y_train = pickle.load(f)  # loading train label data

print(f"train data: {X_train.shape}, {y_train.shape}")

### Loading output

In [ ]:
with open(os.path.join(test_data_path, "test.pickle"), "rb") as f:
    X_test = pickle.load(f)  # loading test image data
with open(os.path.join(test_data_path, "label.pickle"), "rb") as f:
    y_test = pickle.load(f)  # loading test label data
    
print(f"test data: {X_test.shape}, {y_test.shape}")

### Calculating class weights

In [ ]:
class_weights = {}
for i in range(5):
    class_weights[i] = len(y_train) / np.count_nonzero(y_train == i)
print(class_weights)

### Creation of resnet50 model

In [ ]:
input_tensor = Input(shape=(IMG_W, IMG_H, CHAN), dtype='float64')  # input layer
model = ResNet50(weights=None, classes=5, input_tensor=input_tensor)  # resnet50 architecture

model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

### Model training section 

In [ ]:
traing_t0 = time.time()
model.fit(X_train, y_train,
          epochs=10, 
          batch_size=128,
          validation_split=val_split,
          class_weight=class_weights)
train_t = time.time() - traing_t0
model.save(os.path.join(output_data_path, "resnet50_01.model"))  # save trained model
print(f"trained model saved as: {os.path.join(output_data_path, 'resnet50_01.model')}")

### Model testing section

In [ ]:
test_t0 = time.time()
predictions = model.predict(X_test, use_multiprocessing=True)  # predict testing images
results = np.array([np.argmax(prediction)
                    for prediction in predictions])  # process predictions by one high
test_t = time.time() - test_t0
acc = np.count_nonzero(results == y_test) / len(y_test)  # count number of true predicts and calculate accuracy

print(f"accuracy: {acc}, traing_t: {train_t}, test_t: {test_t}")
with open(f"accuracies_resnet50_{IMG_W}x{IMG_H}x{CHAN}_01.out", "w") as f:
    f.write(f"accuracy: {acc}, traing_t: {train_t}, test_t: {test_t}\n")  # save accuracies
    f.flush()